<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Instala las librerías necesarias (solo una vez)
!pip install openml gradio pandas

import openml
import pandas as pd
import gradio as gr

def filtrar_dataset(openml_id, min_value, max_value):
    try:
        # Cargar el dataset por ID
        dataset = openml.datasets.get_dataset(openml_id)
        X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

        # Buscar columna que pueda contener el valor de interés
        print(f"Columnas en el dataset: {X.columns}")

        # Filtrado: el dataset tiene columnas numéricas que se pueden filtrar, por ejemplo, 'gdp' y 'year'
        filtered = X[(y > min_value) & (y < max_value)]

        return filtered.head(20)

    except Exception as e:
        return f"Error: {e}"

# Interfaz de Gradio
gr.Interface(
    fn=filtrar_dataset,
    inputs=[
        gr.Number(label="ID del dataset en OpenML", value=45104),  # Cambié a un dataset económico
        gr.Number(label="Valor mínimo", value=1000),
        gr.Number(label="Valor máximo", value=50000),
    ],
    outputs=gr.Dataframe(label="Resultados filtrados"),
    title="Filtrado de Datasets Económicos (OpenML + Gradio)",
    description="Ingresa un ID de dataset de OpenML con datos económicos y filtra por un rango de valores."
).launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c14480479faf8c3bd7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
